In [1]:
pwd

'/allen/programs/braintv/workgroups/neuralcoding/Parastoo/Mask_RCNN/samples/vesicles'

In [18]:
%%time
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.measure import regionprops
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import datetime
import vesicle

%matplotlib inline 

# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")

import pandas as pd
from PIL import Image
#########################################################################################################

# Configuration

# Dataset directory
# DATASET_DIR = '/allen/programs/braintv/workgroups/neuralcoding/Parastoo/EM_Data'
DATASET_DIR = '/allen/programs/braintv/workgroups/neuralcoding/Parastoo'
# DATASET_DIR = '/allen/programs/braintv/workgroups/neuralcoding/Parastoo/EM_Data'

# DATASET_DIR = '/src/'
# Inference Configuration
config = vesicle.NucleusInferenceConfig()
config.display()


#########################################################################################################
# Notebook preference

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# Only inference mode is supported right now
TEST_MODE = "inference"



def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    fig.tight_layout()
    return ax

#########################################################################################################
# Load test dataset

dataset = vesicle.NucleusDataset()
# dataset.load_nucleus(DATASET_DIR, 'valid')
dataset.load_nucleus(DATASET_DIR, 'TestPilot2')
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

#########################################################################################################
# Load model

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference",
                              model_dir=LOGS_DIR,
                              config=config)
    

weights_path ='/allen/programs/braintv/workgroups/neuralcoding/Parastoo/Mask_RCNN/samples/vesicles/logs/vesicle20210215T1012/mask_rcnn_vesicle_0175.h5'
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)
print('************  Model is loaded  **************')


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1000
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                2.0
IMAGE_RESIZE_MODE              pad64
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

In [19]:
[dataset.image_info[i]['id'] for i in range(len(dataset.image_ids))]

['250_001', '250_002', '250_003', '250_004', '250_005']

In [20]:
ves_nglui = pd.DataFrame(columns = ['img_id', 'area_vox', 'ctr_pts', 'p_score'])

# df_path = '/allen/programs/braintv/workgroups/neuralcoding/Parastoo/Pilot_Logs/meta_data_202102280011.csv'
# ves_nglui = pd.read_csv(df_path, index_col=0)

In [21]:
ves_nglui

,img_id,area_vox,ctr_pts,p_score


In [22]:
dir_path = '/allen/programs/braintv/workgroups/neuralcoding/Parastoo/TestPilot2/'
ves_path = '/allen/programs/braintv/workgroups/neuralcoding/Parastoo/TestPilot2/meta_data_{}.csv'.format(datetime.datetime.now().strftime("%Y%m%d%H%M"))
# error_path = '/allen/programs/braintv/workgroups/neuralcoding/Parastoo/PilotData/error_log_{}.csv'.format(datetime.datetime.now().strftime("%Y%m%d%H%M"))

In [23]:
# ves_path = '/allen/programs/braintv/workgroups/neuralcoding/Parastoo/Pilot_Logs/meta_data_202102230044.csv'

In [24]:
def gen_ctr_pts(pred_masks):
    
    ctr_pts = []
    for i in range(pred_masks.shape[2]):
        region = regionprops(pred_masks[:, :, i])
        try:
            y, x = region[0].centroid
            ctr_pt = [x, y, 0]
            ctr_pts.append(ctr_pt)
        except IndexError:
            return;
    return ctr_pts

def make_results_df(results, img_id):
    
    # define the dataframe and attributes
    df = pd.DataFrame(columns = ['img_id', 'area_vox', 'ctr_pts', 'p_score'])
    pred_masks = results['masks']
    scores = results['scores']
    masks2 = np.reshape(pred_masks > .5, (-1, pred_masks.shape[-1])).astype(np.float32)
    area = np.sum(masks2, axis=0)
    df.p_score = scores
    df.area_vox = area
    df.ctr_pts = gen_ctr_pts(pred_masks.astype(int))
    df.img_id = img_id
    
    return df

    

def save_masks(pred_masks, img_id, dir_path = dir_path, file_exists=True):
    
    
    if file_exists==False:
        os.mkdir(dir_path + img_id + '/' + 'masks')
        pred_path = dir_path + img_id + '/' + 'masks'
    else:
        pred_path = dir_path + img_id + '/' + 'masks'
        
    for i in range(pred_masks.shape[2]):
        mask = pred_masks[:, :, i]
        pil_mask = Image.fromarray(mask)
        pil_mask.save('{}/{}_{}.png'.format(pred_path,img_id,i))
        

for image_id in dataset.image_ids:
    print(image_id)
    img_id = dataset.image_info[image_id]['id']
    print(img_id)
    if img_id not in np.unique(ves_nglui.img_id):
        image = dataset.load_image(image_id)

        # Detect objects
        r = model.detect([image], verbose=0)[0]

        try:
            try:
                df = make_results_df(r, img_id)

            except FileNotFoundError:
                 df = make_results_df(r, img_id, file_exists=False)

            ves_nglui = ves_nglui.append(df)
            ves_nglui.to_csv(ves_path)
            save_masks(r['masks'], img_id)


        except ValueError:
            pass


0
250_001
1
250_002
2
250_003
3
250_004
4
250_005


In [59]:
ves_nglui

,img_id,area_vox,ctr_pts,p_score
0,250_001,158.0,"[27.917721518987342, 98.25949367088607, 0]",0.999047
1,250_001,131.0,"[271.7022900763359, 129.4351145038168, 0]",0.998346
2,250_001,181.0,"[161.74585635359117, 130.27624309392266, 0]",0.997797
3,250_001,109.0,"[28.844036697247706, 80.19266055045871, 0]",0.994334
4,250_001,145.0,"[288.9448275862069, 120.99310344827586, 0]",0.989053
...,...,...,...,...
11,250_100,94.0,"[62.5, 270.70212765957444, 0]",0.749756
12,250_100,81.0,"[203.3827160493827, 111.95061728395062, 0]",0.712849
13,250_100,97.0,"[187.0721649484536, 46.69072164948454, 0]",0.702013
14,250_100,147.0,"[9.517006802721088, 255.7074829931973, 0]",0.545646


In [60]:
ves_nglui.to_csv(ves_path)

In [57]:
# DateTime.Now.ToFileTime()
datetime.datetime.now().strftime("%Y%m%d-%H%M")

'20210223-0025'

In [1]:
su

NameError: name 'pd' is not defined